### Imports

In [5]:
import os, sys
from bs4 import BeautifulSoup
import pandas as pd
import requests
import splinter

### NASA Mars News

In [69]:
# request url for mars news api
json_url=('https://mars.nasa.gov/api/v1/news_items/'
          '?page=0&per_page=10'
          '&order=publish_date+desc%2Ccreated_at+desc'
          '&search='
          '&category=19%2C165%2C184%2C204'
          '&blank_scope=Latest')

response = requests.get(json_url)
if response.status_code == 200:
    a=response.json()
    news_item = a.get('items')[0]
    news_title = news_item.get('title')
    news_description = news_item.get('description')
    news_p = BeautifulSoup(news_item.get('body'), 'html.parser').p.get_text()
    pass

In [70]:
print("Title:", news_title, end="\n\n")
print("Description:", news_description, end="\n\n")
print("Blurb:", news_p, end="\n\n")

Title: NASA's Mars 2020 Comes Full Circle

Description: Aiming to pinpoint the Martian vehicle's center of gravity, engineers took NASA's 2,300-pound Mars 2020 rover for a spin in the clean room at JPL. 

Blurb: ​Engineers took NASA's Mars 2020 for a spin on Aug. 29, 2019. The 2,300-pound (1,040-kilogram) Martian vehicle was rotated clockwise and counterclockwise at about 1 revolution per minute on what is called a spin table in the clean room of the Spacecraft Assembly Facility at NASA's Jet Propulsion Laboratory in Pasadena, California. (The rotation was speeded up in the video above.) The engineers were looking for the rover's center of gravity, or the point at which weight is evenly dispersed on all sides.



### JPL Mars Space Images - Featured Image

In [42]:
# NOTE: some of the featured images didn't have associated 'hires' links; 
#   so I used the default wallpaper link

url=('https://www.jpl.nasa.gov/spaceimages/'
     '?search='
     '&category=Mars')

response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser').\
        find('article', 'carousel_item')
    featured_image_url = 'https://www.jpl.nasa.gov' + soup.attrs['style'].split("'")[1]
    pass

In [43]:
print("Image URL:", featured_image_url)

Image URL: https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17449-1920x1200.jpg


### Mars Weather

In [44]:
url=('https://twitter.com/marswxreport'
     '?lang=en')

brws = splinter.Browser("chrome", executable_path="/home/burned/.local/bin/chromedriver")
brws.visit(url)
html = brws.html
brws.quit()

soup = BeautifulSoup(html, 'html.parser')
for s in soup.find_all('div', 'tweet'):
    # this is *dumb*... should be using twitter's retarded api (which is also *dumb*)
    #   ... so maybe this is less dumb?
    txt=s.get_text()
    if txt.find('InSight sol') >= 0:
        mars_weather = txt[txt.find('InSight sol'):txt.find('hPa')+3]
        break

In [45]:
print("Mars Weather:", mars_weather)

Mars Weather: InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)
winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)
pressure at 7.50 hPa


### Mars Facts

In [53]:
url='https://space-facts.com/mars/'
df = pd.read_html(url)
df[0].set_index('Mars - Earth Comparison', inplace=True)
mars_facts_table = df[0].to_html()

In [60]:
print(mars_facts_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
    <tr>
      <th>Mars - Earth Comparison</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-153 to 20 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


### Mars Hemispheres

In [5]:
url=('https://astrogeology.usgs.gov/search/results'
     '?q=hemisphere+enhanced'
     '&k1=target'
     '&v1=Mars')


